In [25]:
# get the code
import sys
import os
import re
import matplotlib.pyplot as plt
# use seaborn plotting defaults
import seaborn as sns; sns.set()
sys.path.append('../code')

# import package functions
from script_utils_CNV import get_CNVconfig, show_output

######## need to find the chained assignment!!!
pd.set_option('mode.chained_assignment', None)

# HOME
home = '/Users/mahtin'
home = '/Users/martinscience'


# standard paths
static = os.path.join(home, "Dropbox/Icke/Work/static")
tooldata = os.path.join(home, "Dropbox/Icke/Work/somVar/tooldata")
testdata = os.path.join(home,"Dropbox/Icke/Work/somVar/testdata")
PON_path = os.path.join(static, "PON/HAEv7_hg38_NovaSeq")


cnvdata = os.path.join(tooldata, "myCNVdata")
output_path = os.path.join(cnvdata, "output")
plot_path = os.path.join(cnvdata, "plot")
fig_path = os.path.join(cnvdata, "figures")

### get the config
+ use the get_CNVconfig util function to update the general configs with the appropriate paths

In [26]:
path_config = dict(
        mawk_path="../shell",
        cov_path=os.path.join(output_path, "pile2CNV"),   # path containing rawcov.gz files for this sample
        snp_path=os.path.join(output_path, "pile2CNV"),   # path containing snp files for this sample
        bed_file=os.path.join(static, "bed_files/SureSelect/hg38/SS_HAEv7_hg38_Padded.bed"),
        genome_split_path=os.path.join(static, "genome/gatk/hg38/split"),
        gc_split_path=os.path.join(static, "genome/gatk/hg38/split"),
        genmap_split_path=os.path.join(static, "annotation/genmap/hg38/split"),
        PON_path = PON_path,
    )
CNVconfig = get_CNVconfig(
    "../config/config_CNV.yaml", 
    local_config=path_config)

### load cnv data

In [27]:
sample = "03_A-B"
cnv_df = pd.read_csv(os.path.join(output_path, f"CNV/{sample}.cnv.snp.gz"), sep="\t", compression="gzip")
cnv_df

,Chr,Pos,ExonPos,FullExonPos,SNPdensity,SNPdensityDiff,map30_0,map50_0,map75_1,map100_2,Depth1,VAF1,absVAF1,offVAF1,Depth2,VAF2,absVAF2,offVAF2,PONVAF,PONDepth,log2ratio1,log2ratio1_mean,log2ratio2,log2ratio2_mean,PONcov_mean,PONcov_median,PONcov_std,GCratio,fallSNP
0,chr1,1043223,52185,52185,0.000091,1.866617e-07,1.0,1.0,1.0,1.0,155.0,0.256940,0.486120,-0.486120,250.0,0.233060,0.533880,-0.533880,0.3592,1826.0,0.126618,0.008411,0.425306,0.103154,84.981868,84.204021,18.027826,0.676000,0.000107
1,chr1,1185051,72189,72189,0.000472,9.016908e-08,1.0,1.0,1.0,1.0,245.0,0.555952,0.111904,0.111904,410.0,0.515109,0.030218,0.030218,0.0000,0.0,-0.183949,-0.179762,0.192468,0.016765,120.477314,120.022251,16.911436,0.610000,0.000000
2,chr1,1212042,77575,77575,0.000379,6.206060e-08,1.0,1.0,1.0,1.0,235.0,0.416308,0.167384,-0.167384,391.0,0.533224,0.066448,0.066448,0.2258,2679.0,-0.070702,-0.147116,0.279253,-0.045892,100.753978,100.489776,16.067632,0.702000,0.000281
3,chr1,1213065,77866,77866,0.000424,1.262299e-07,1.0,1.0,1.0,1.0,305.0,0.496601,0.006798,-0.006798,385.0,0.494203,0.011594,-0.011594,0.0000,0.0,-0.189284,-0.139325,-0.201967,-0.018217,131.224536,126.887062,19.028327,0.705000,0.000000
4,chr1,1288032,92921,92921,0.000157,1.232839e-08,1.0,1.0,1.0,1.0,145.0,0.541033,0.082066,0.082066,222.0,0.541565,0.083130,0.083130,0.0000,0.0,-0.300211,-0.170442,-0.002328,0.001975,75.867306,74.487764,11.029186,0.705962,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26253,chrX,156001308,3319393,92565888,0.000545,1.331408e-07,1.0,0.5,0.5,0.5,42.0,0.561395,0.122790,0.122790,51.0,0.507648,0.015296,0.015296,0.1806,404.0,-0.430992,-0.071145,-0.352875,-0.046761,35.953309,32.361011,14.904314,0.625116,0.000323
26254,chrX,156003433,3320286,92566781,0.000710,1.725148e-07,1.0,1.0,1.0,0.5,186.0,0.467693,0.064614,-0.064614,283.0,0.482290,0.035420,-0.035420,0.3160,2984.0,-0.809021,-0.176001,-0.546104,-0.108923,92.789091,102.935169,34.027976,0.602356,0.000737
26255,chrX,156009834,3323422,92569917,0.000479,2.541281e-05,1.0,1.0,0.5,0.5,15.0,0.304252,0.391496,-0.391496,39.0,0.216547,0.566906,-0.566906,0.1041,192.0,-0.947982,-0.203235,-0.229637,-0.075847,32.530616,31.814244,23.051767,0.613251,0.000164
26256,chrX,156010159,3323747,92570242,0.000619,6.435463e-05,0.5,1.0,1.0,1.0,54.0,0.426474,0.147052,-0.147052,102.0,0.380197,0.239606,-0.239606,0.1764,731.0,-0.613110,-0.273128,-0.064634,-0.097654,35.569818,32.347916,18.333052,0.610000,0.000359


In [30]:
def write_ASCAT(cnv_df, sample="", outpath="", mock_germline=True):
    """
    for a sample XX_A-B it writes into outpath/XX/:
    XX_A-B_A_logr_tumor.tsv
    XX_A-B_B_logr_tumor.tsv
    XX_A-B_A_baf_tumor.tsv
    XX_A-B_B_baf_tumor.tsv
    XX_A-B_baf_normal.tsv
    XX_A-B_logr_normal.tsv


    sample comes in shape sample_tumor-normal
    """
    s = sample.split("_")
    sample_name = s[0]
    tumor_name = s[1].split("-")[0]
    normal_name = s[1].split("-")[1]

    # base_file XX_A-B
    base_file = os.path.join(outpath, sample)

    # change cols
    cnv_df.loc[:, "Chr"] = cnv_df["Chr"].str.replace("chr", "")

    # make mock germline
    mock_df = cnv_df.loc[:, ["Chr", "Pos", "VAF1", "log2ratio1_mean"]].rename(
        dict(
            Chr="chr",
            Pos="pos",
        )
    )
    mock_df.loc[:, "VAF1"] = 0.5
    mock_df.loc[:, "log2ratio1_mean"] = 0

    # #### VAF

    vaf_cols = [col for col in cnv_df.columns if col.startswith("VAF")]

    normal_cols = {"Chr": "chrs", "Pos": "pos", vaf_cols[0]: sample}
    normal_baf_file = f"{base_file}_{normal_name}_baf.tsv"
    cnv_df.loc[:, normal_cols.keys()].rename(normal_cols, axis=1).to_csv(
        normal_baf_file, sep="\t"
    )

    tumor_cols = {"Chr": "chrs", "Pos": "pos", vaf_cols[1]: sample}
    tumor_baf_file = f"{base_file}_{tumor_name}_baf.tsv"
    cnv_df.loc[:, tumor_cols.keys()].rename(tumor_cols, axis=1).to_csv(
        tumor_baf_file, sep="\t"
    )
    if mock_germline:
        # store the mock_df as fake germline sample
        mock_baf_file = f"{base_file}_germline_baf.tsv"
        mock_df.loc[:, normal_cols.keys()].rename(normal_cols, axis=1).to_csv(
            mock_baf_file, sep="\t"
        )

    # log2ratio
    log2_pat = re.compile(r"log2ratio[0-9]+_mean$")
    log_cols = [col for col in cnv_df.columns if re.match(log2_pat, col)]

    normal_cols = {"Chr": "chrs", "Pos": "pos", log_cols[0]: sample}
    normal_log_file = f"{base_file}_{normal_name}_logr.tsv"
    cnv_df.loc[:, normal_cols.keys()].rename(normal_cols, axis=1).to_csv(
        normal_log_file, sep="\t"
    )

    tumor_cols = {"Chr": "chrs", "Pos": "pos", log_cols[1]: sample}
    tumor_log_file = f"{base_file}_{tumor_name}_logr.tsv"
    cnv_df.loc[:, tumor_cols.keys()].rename(tumor_cols, axis=1).to_csv(
        tumor_log_file, sep="\t"
    )
    if mock_germline:
        # store the mock_df as fake germline sample
        mock_log_file = f"{base_file}_germline_logr.tsv"
        mock_df.loc[:, normal_cols.keys()].rename(normal_cols, axis=1).to_csv(
            mock_log_file, sep="\t"
        )

    show_output(
        f"ASCAT output written to {base_file}_[baf/logr]_[tumor|normal].tsv",
        color="success",
    )
    return cnv_df

In [32]:
_ = write_ASCAT(cnv_df, sample="03_A-B", outpath=os.path.join(output_path, "CNV/ASCAT/03"))

ASCAT output written to /Users/martinscience/Dropbox/Icke/Work/somVar/tooldata/myCNVdata/output/CNV/ASCAT/03/03_A-B_[baf/logr]_[tumor|normal].tsv


In [22]:
TN_list = ['03_A-B', '04_A-B']

In [23]:
[f"{tn.split('_')[0]}/{tn}_{t}" for tn in TN_list for t in tn.split("_")[1].split("-")]

['03/03_A-B_A', '03/03_A-B_B', '04/04_A-B_A', '04/04_A-B_B']

In [24]:
[f"{tn.split('_')[0]}/{tn}" for tn in TN_list]

['03/03_A-B', '04/04_A-B']